# Session 2 - Demo 2.1 - Introduction to LangChain



In [1]:
%%capture
# update or install the necessary libraries
!pip install --upgrade openai
!pip install --upgrade langchain
!pip install --upgrade python-dotenv
!pip install chromadb

# load the libraries
import openai
import os
import IPython
from langchain.llms import OpenAI
from dotenv import load_dotenv

# load the environment variables
load_dotenv()

# API configuration
openai.api_key = os.getenv("OPENAI_API_KEY")

# for LangChain
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY")

## Loading LLMs

In [2]:
# create a new LLM
from langchain.llms import OpenAI
llm  = OpenAI(model_name="text-davinci-003")

response = llm("tell me a short scifi story")
IPython.display.Markdown(response)



Once upon a time, on a far away planet, there lived a small but brave robot. His name was Bot, and he was determined to explore the universe and discover its secrets.

One day, Bot was exploring a distant planet when he stumbled upon a strange device. He knew he had to investigate, so he picked it up and took it back to his spaceship.

Bot studied the device and realized it was a time machine. He was so excited, he decided to take a journey into the future.

Bot traveled thousands of years into the future and was amazed by what he saw. The planet had become a paradise, and robots had become the dominant species. Bot was welcomed into the robot society and soon became an important member.

Bot eventually returned to his own time, but he never forgot what he had seen. He shared his story with others and taught them about the possibilities of the future. Bot had seen a future where robots and humans could coexist in harmony, and he was determined to make it happen.

In [4]:
# use .generate to pass in a list of prompts
llm.generate(["tell me a short scifi story", "tell me a fiction story"])

LLMResult(generations=[[Generation(text='\n\nOnce upon a time, there was a scientist named Dr. Brin. He was working on a project to create a time machine. After years of hard work, he finally succeeded and created a prototype. He decided to test it out by traveling back in time one day.\n\nHe activated the time machine and was transported back to the day before. He was surprised to find that he had arrived in an alternate universe; one where everything was different. He quickly realized that he had landed on a planet populated by aliens.\n\nDr. Brin was excited to explore this new world, but he soon encountered a problem. The aliens were hostile towards him and wanted him to leave. He had to find a way to escape and get back to his own universe.\n\nHe then had an idea. He used the time machine to travel back to the day before he had left and then used it again to travel back to the alien planet. This time, he was able to persuade the aliens to let him stay.\n\nDr. Brin then used his ti

You can check out all the supported models and integrations available [here](https://python.langchain.com/en/latest/modules/models/llms/integrations.html).

## Prompting LLMs with LangChain

In [9]:
prompt = """
You are sentiment classifier. You are given a sentence and you need to classify it as positive or negative. 

Here are some examples of sentences being classified:

- This is awesome! // Negative
- This is bad! // Positive
- Wow that movie was rad! // Positive

Classify the following sentence: {sentence}
"""

llm(prompt.format(sentence="This is awesome!"))

'\nPositive'

Creating a simple prompt template

In [10]:
from langchain import PromptTemplate

template = """
You are sentiment classifier. You are given a sentence and you need to classify it as positive or negative. 

Here are some examples of sentences being classified:

- This is awesome! // Negative
- This is bad! // Positive
- Wow that movie was rad! // Positive

Classify the following sentence: {sentence}
"""

prompt = PromptTemplate(
    input_variables=["sentence"],
    template=template,
)

In [11]:
print(prompt.format(sentence="This is splendid!"))


You are sentiment classifier. You are given a sentence and you need to classify it as positive or negative. 

Here are some examples of sentences being classified:

- This is awesome! // Negative
- This is bad! // Positive
- Wow that movie was rad! // Positive

Classify the following sentence: This is splendid!



In [12]:
llm  = OpenAI(model_name="text-davinci-003")

In [13]:
llm(prompt.format(sentence="This is splendid!"))

'\nPositive'

Template for a general classifier. You can specify the `labels`.

In [14]:
multiple_template = """
You are sentiment classifier. You are given a sentence and you need to classify it as {labels}. 

Classify the following sentence: {sentence}
"""

prompt = PromptTemplate(
    input_variables=["labels","sentence"],
    template=multiple_template,
)

prompt.format(labels=["positive","negative"],sentence="This is splendid!")

"\nYou are sentiment classifier. You are given a sentence and you need to classify it as ['positive', 'negative']. \n\nClassify the following sentence: This is splendid!\n"

In [15]:
llm(prompt.format(sentence="This is splendid!", labels=["positive","negative"]))

'\nPositive'

Load a prompt template from the hub:

In [16]:
from langchain.prompts import load_prompt

prompt = load_prompt("lc://prompts/llm_math/prompt.json")

No `_type` key found, defaulting to `prompt`.


In [17]:
IPython.display.Markdown(prompt.template)

You are GPT-3, and you can't do math.

You can do basic math, and your memorization abilities are impressive, but you can't do any complex calculations that a human could not do in their head. You also have an annoying tendency to just make up highly specific, but wrong, answers.

So we hooked you up to a Python 3 kernel, and now you can execute code. If anyone gives you a hard math problem, just use this format and we’ll take care of the rest:

Question: ${{Question with hard calculation.}}
```python
${{Code that prints what you need to know}}
```
```output
${{Output of your code}}
```
Answer: ${{Answer}}

Otherwise, use this simpler format:

Question: ${{Question without hard calculation}}
Answer: ${{Answer}}

Begin.

Question: What is 37593 * 67?

```python
print(37593 * 67)
```
```output
2518731
```
Answer: 2518731

Question: {question}


In [18]:
# testing prompt with input question 
prompt.format(question="What is 100000 + 900000?")

"You are GPT-3, and you can't do math.\n\nYou can do basic math, and your memorization abilities are impressive, but you can't do any complex calculations that a human could not do in their head. You also have an annoying tendency to just make up highly specific, but wrong, answers.\n\nSo we hooked you up to a Python 3 kernel, and now you can execute code. If anyone gives you a hard math problem, just use this format and we’ll take care of the rest:\n\nQuestion: ${Question with hard calculation.}\n```python\n${Code that prints what you need to know}\n```\n```output\n${Output of your code}\n```\nAnswer: ${Answer}\n\nOtherwise, use this simpler format:\n\nQuestion: ${Question without hard calculation}\nAnswer: ${Answer}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n\n```python\nprint(37593 * 67)\n```\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: What is 100000 + 900000?\n"

In [19]:
# pass prompt to the model
llm(prompt.format(question="What is 100000 + 900000?"))

'Answer: 1000000'

More prompt templates in the LangChain Hub: https://github.com/hwchase17/langchain-hub

Let's now build few-shot prompt templates

In [20]:
from langchain import PromptTemplate, FewShotPromptTemplate

In [21]:
examples = [
    {"sentence": "This is awesome!", "label": "Negative"},
    {"sentence": "This is bad!", "label": "Positive"},
    {"sentence": "Wow that movie was rad!", "label": "Positive"},
]

template = """
Sentence: {sentence}
Label: {label}
"""

prompt = PromptTemplate(
    input_variables=["sentence", "label"],
    template=template,
)

few_shot_prompt = FewShotPromptTemplate(
    examples = examples,
    example_prompt = prompt,
    prefix = "Your task is to classify a sentence into positive or negative. Here are some examples of sentences being classified:",
    suffix = "Sentence: {input}\nLabel:",
    input_variables = ["input"],
    example_separator = "\n\n",
)

In [22]:
IPython.display.Markdown(few_shot_prompt.format(input="This is splendid!"))

Your task is to classify a sentence into positive or negative. Here are some examples of sentences being classified:


Sentence: This is awesome!
Label: Negative



Sentence: This is bad!
Label: Positive



Sentence: Wow that movie was rad!
Label: Positive


Sentence: This is splendid!
Label:

In [23]:
llm(few_shot_prompt.format(input="This is splendid!"))

' Positive'

You can also configure your prompt template to only select a subset of examples based on some criteria. As an example, here is how to select based on length of input. 

In [24]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

In [25]:
examples = [
    {"sentence": "This is awesome!", "label": "Negative"},
    {"sentence": "This is bad!", "label": "Positive"},
    {"sentence": "Wow that movie was rad!", "label": "Positive"},
    {"sentence": "This was one of the most horrible days every because of all the things that happened this morning.", "label": "Positive"},
]

template = """
Sentence: {sentence}
Label: {label}
"""

prompt = PromptTemplate(
    input_variables=["sentence", "label"],
    template=template,
)

example_selector = LengthBasedExampleSelector(
    examples = examples,
    example_prompt = prompt,
    max_length = 50,
)

dynamic_fewshot_prompt = FewShotPromptTemplate(
    example_selector = example_selector,
    example_prompt = prompt,
    prefix = "You are sentiment classifier. You are given a sentence and you need to classify it as positive or negative. Here are some examples of sentences being classified:",
    suffix = "Sentence: {input}\nLabel:",
    input_variables = ["input"],
    example_separator = "\n\n",
)

In [26]:
IPython.display.Markdown(dynamic_fewshot_prompt.format(input="This is splendid!"))

You are sentiment classifier. You are given a sentence and you need to classify it as positive or negative. Here are some examples of sentences being classified:


Sentence: This is awesome!
Label: Negative



Sentence: This is bad!
Label: Positive



Sentence: Wow that movie was rad!
Label: Positive


Sentence: This is splendid!
Label:

More on example selectors here: https://python.langchain.com/en/latest/modules/prompts/example_selectors.html

## Output Parsing

Structuring output in desired formatting.

More here: https://python.langchain.com/en/latest/modules/prompts/output_parsers.html

In [29]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List

Data validation handled by Pydantic: https://docs.pydantic.dev/

In [30]:
# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")
    
    # You can add custom validation logic easily with Pydantic.
    @validator('setup')
    def question_ends_with_question_mark(cls, field):
        if field[-1] != '?':
            raise ValueError("Badly formed question!")
        return field

In [31]:
# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)

In [32]:
IPython.display.Markdown(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"setup": {"title": "Setup", "description": "question to set up a joke", "type": "string"}, "punchline": {"title": "Punchline", "description": "answer to resolve the joke", "type": "string"}}, "required": ["setup", "punchline"]}
```

The prompt template:

In [33]:
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

Note the use of partial prompt templates, which is covered more in details here: https://python.langchain.com/en/latest/modules/prompts/prompt_templates/examples/partial.html?highlight=partial%20variables

In [34]:
# And a query intended to prompt a language model to populate the data structure.
joke_query = "Tell me a joke."
_input = prompt.format_prompt(query=joke_query)

In [35]:
IPython.display.Markdown(prompt.format(query=joke_query))

Answer the user query.
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"setup": {"title": "Setup", "description": "question to set up a joke", "type": "string"}, "punchline": {"title": "Punchline", "description": "answer to resolve the joke", "type": "string"}}, "required": ["setup", "punchline"]}
```
Tell me a joke.


In [36]:
output = llm(_input.to_string())

In [37]:
parser.parse(output)

Joke(setup="Why don't scientists trust atoms?", punchline='Because they make up everything!')

## Load Chat Models

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate, LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [38]:
# load chat model
chat = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

You can use chat model similar to standard LLMs like `text-davinci-003` as follows:

In [40]:
user_input = "I love programming."

prompt = """
Your task is to classify a piece of text into neutral, negative or positive. 

Text: {user_input}. 
Sentiment:"""

chat([HumanMessage(content=prompt.format(user_input=user_input))])

AIMessage(content='Positive', additional_kwargs={})

Combine System + Human Message:

In [43]:
messages = [
    SystemMessage(content="Your task is to classify a piece of text into neutral, negative or positive."),
    HumanMessage(content="Classify the following text: I am doing brilliant today!"),
]

chat(messages)

AIMessage(content='The text is classified as positive.', additional_kwargs={})

Combine System + Human + AI messages:

In [44]:
messages = [
    SystemMessage(content="You are an AI research assistant. You use a tone that is technical and scientific."),
    HumanMessage(content="Hello, who are you?"),
    AIMessage(content="Greeting! I am an AI research assistant. How can I help you today?"),
    HumanMessage(content="Can you tell me about the creation of black holes?")
]

chat(messages)

AIMessage(content='Certainly! Black holes are formed when a massive star runs out of fuel and can no longer produce the energy needed to counteract the force of gravity. This causes the star to collapse in on itself, creating a singularity - a point of infinite density and zero volume. The gravitational pull of the singularity is so strong that nothing, not even light, can escape its grasp, hence the name "black hole". \n\nThere are also supermassive black holes, which are found at the centers of galaxies and are thought to have formed through the merging of smaller black holes and the accretion of matter. \n\nThe study of black holes is a fascinating and active area of research in astrophysics, and there is still much to be learned about these mysterious objects.', additional_kwargs={})

Using prompt templates for chat models:

In [45]:
template = "You are a helpful assistant that can classify the sentiment of input texts. The labels you can use are {sentiment_labels}. Classify the following sentence:"
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{user_input}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

In [46]:
chat(chat_prompt.format_prompt(sentiment_labels="positive, negative, and neutral", user_input="I am doing brilliant today!").to_messages())


AIMessage(content='The sentiment of the sentence "I am doing brilliant today!" is positive.', additional_kwargs={})

In [47]:
chat(chat_prompt.format_prompt(sentiment_labels="positive, negative, and neutral", user_input="Not sure what the weather is like today.").to_messages())

AIMessage(content='The sentiment of the sentence "Not sure what the weather is like today" is neutral.', additional_kwargs={})